In [22]:
import pandas as pd
import numpy as np
import csv
import os
import json
path ="metadata.csv" 
df = pd.read_csv(path, encoding = 'utf-8')

In [26]:
#directories = ["biorxiv_medrxiv","comm_use_subset","custom_license","noncomm_use_subset"]
directories = ["biorxiv_medrxiv"]
json_array = []
for d in directories:
    for file in os.listdir(f"{d}/{d}"):
        #print(file)
        file_path = f"{d}/{d}/{file}"
        j = json.load(open(file_path, "rb"))
        json_array.append(j)
#finds json files and saves to an array

In [27]:
len(json_array)

885

In [28]:
#search_keys 
for key in json_array[0]:
    print(key)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [35]:
raw_abstract = (json_array[0]['abstract'])
raw_metadata = (json_array[0]['metadata'])
raw_bodytext = (json_array[0]['body_text'])
raw_bib_entries = (json_array[0]['bib_entries'])
raw_ref = (json_array[0]['ref_entries'])
raw_back_mater = (json_array[0]['back_matter'])

In [41]:
##------------------------------------------------------------##
raw_bodytext
body_full_text = []
for key in raw_bodytext:
    body_full_text.append(key['text'] + '\n\n')
    #print(key['text'])

In [43]:
#body_full_text

In [ ]:
#needs more structuring.
#Tasks as per Kaggle:- what is known about transmission/incubation and environmental stability?
###Points to consider::--- Extract meaning(NLP?) Terms displayed in Tasks may be called something else in the dataset/research papers...